In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from geopy.distance import geodesic
import requests

In [ ]:

def get_place_types(query, latitude, longitude, radius=10000):
    """
    Fetch Google Place types for a specific business query (e.g., Walmart).
    """
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location": f"{latitude},{longitude}",
        "radius": radius,
        "keyword": query,
        "key": API_KEY
    }
    
    response = requests.get(url, params=params)
    data = response.json()

    # Print API response for debugging
    print("API Response:", data)

    if data.get("status") != "OK":
        print(f"Error: {data.get('error_message', 'Unknown error')}")
        return []

    if "results" in data:
        for place in data["results"]:
            print(f"Name: {place['name']}")
            print(f"Types: {place['types']}\n")

# Example usage: Find Walmart types in Atlanta


In [22]:
get_place_types("Wells fargo", 33.7490, -84.3880)

API Response: {'html_attributions': [], 'next_page_token': 'AUy1YQ3pvknpd5M3sjPmz_v1GI2oWBkgb2R49C7wumcr7B8YpYaF-sQ7HJpkm4dZ0zLSaG3T5hVR-GZDTUFnacLRnnGA5TOIArr-CDsshil55k94vuQITjCSJI87OeWN0vFDIFkJUWiUenzuhwUwA3dBLlynQbjPOYv4OrpVZ0X8FCr0uZA1D663FQsgLaC1Cox7ZiyQz-O0k6aYKmB_ydvXW0i9bk3ju7qoA_xbQePMORShgniypWOJUW57NvEt4pHCYZtsU-_ywyw9VIeI8vZ5wLmhCQ545jkwIPdW8UqnvfoMeN7AX8WvLBQ5FAX4dQvk66wkmHcRZ8x0q3ebmk-h7SAv9Ige9XrWtjGx_BIeDMUmse9iZA1OWhfTgz6J7UCj6XveSFuoCidD2uw8c-VuYLy2Xg2DpttC21s9ktsZwZW4vsIl_--4eCp6pBh6-w', 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 33.8157978, 'lng': -84.3097016}, 'viewport': {'northeast': {'lat': 33.81709872989272, 'lng': -84.30846092010728}, 'southwest': {'lat': 33.81439907010728, 'lng': -84.31116057989273}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bank-71.png', 'icon_background_color': '#909CE1', 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/bank-intl_pinlet', 'name': '

In [ ]:
DESIRABLE_AMENITIES = {
    "national_big_box_store": {"group": 1, "google_type": "department_store", "store_names": ["Walmart", "Target", "Costco", "BJ's", "Sam's Club"]},
    "retail_store": {"group": 2, "google_type": "clothing_store"},
    "grocery_store": {"group": 1, "google_type": "grocery_or_supermarket"},
    "restaurant": {"group": 2, "google_type": "retaurant"},
    "hospital": {"group": 1, "google_type": "hospital"},
    "medical_clinic": {"group": 1, "google_type": "doctor"},
    "pharmacy": {"group": 1, "google_type": "pharmacy"},
    "licensed_childcare": {"group": 1},
    "technical_college": {"group": 2},
    "school": {"group": 1},
    "town_square": {"group": 1},
    "community_center": {"group": 1},
    "large_public_park": {"group": 1},
    "small_public_park": {"group": 2},
    "library": {"group": 1, "google_type": "library"},
    "fire_police_station": {"group": 2},
    "bank": {"group": 2, , "google_type": "bank"}},
    "place_of_worship": {"group": 2, "google_type": "place_of_worship"},
    "post_office": {"group": 2, "google_type": "post_office"},
}

UNDESIRABLE_ACTIVITIES = [
    "junkyard", "dump", "landfill", "chemical_factory", "abandoned_building",
    "deteriorated_housing", "hazardous_site", "auto_repair_station", "dry_cleaner"
]


industrial_facility
waste_management
farm

In [ ]:
def assign_points(distance, group):
    """
    Dynamically assigns points based on amenity group and distance.
    """
    if group == 1:  
        if distance <= 0.5:
            return 2.5
        elif distance <= 1:
            return 2
        elif distance <= 1.5:
            return 1.5
    elif group == 2:  
        if distance <= 0.5:
            return 2
        elif distance <= 1:
            return 1.5
        elif distance <= 2.5:
            return 1
    return 0 

In [ ]:
# Compute distance between two locations
def get_distance(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).miles

# Compute desirability score
def calculate_site_score(site_lat, site_lon, amenities_df, undesirable_df):
    total_points = 0
    deductions = 0

    for _, row in amenities_df.iterrows():
        amenity_type = row["amenity_type"]
        amenity_lat, amenity_lon = row["latitude"], row["longitude"]
        
        if amenity_type in DESIRABLE_AMENITIES:
            distance = get_distance(site_lat, site_lon, amenity_lat, amenity_lon)
            group = DESIRABLE_AMENITIES[amenity_type]["group"]
            points = assign_points(distance, group)
            total_points += points

    for _, row in undesirable_df.iterrows():
        undesired_lat, undesired_lon = row["latitude"], row["longitude"]
        distance = get_distance(site_lat, site_lon, undesired_lat, undesired_lon)
        if distance <= 0.25:
            deductions += 2

    final_score = max(0, total_points - deductions)
    return {"total_points": total_points, "deductions": deductions, "final_score": final_score}

# Example site coordinates
site_latitude = 33.7490
site_longitude = -84.3880

# Load datasets
amenities_df = pd.read_csv("../../data/processed/desirable_amenities.csv")
undesirable_df = pd.read_csv("../../data/processed/undesirable_activities.csv")

# Compute score
score_info = calculate_site_score(site_latitude, site_longitude, amenities_df, undesirable_df)

print("Total Points (Desirable):", score_info["total_points"])
print("Deductions (Undesirable):", score_info["deductions"])
print("Final Score:", score_info["final_score"])